In [2]:
import requests as r
import sqlite3
import lxml
from bs4 import BeautifulSoup
import math
import time
import pprint
import threading


In [3]:
# index
index = "https://www.104.com.tw"

# job page list href (page1)
# https://www.104.com.tw/jobbank/joblist/auto_joblist.cfm?auto=1&jobsource=n104bank1&ro=0&jobcat=2007000000&order=2&asc=0&page=1&psl=N_A

# job href selector
# soup.select('div.jobname_summary')[0].select('a')[0]['href']

In [40]:
# create table & drop table
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
    try:
        c.execute('drop table job104')
        print('drop table job104')
    except:
        c.execute('''
            CREATE TABLE job104(
              jobID INTEGER PRIMARY KEY AUTOINCREMENT,
              title TEXT NOT NULL,
              href TEXT NOT NULL,
              times INTEGER,
              info TEXT)''')
        print('create table job104')

create table job104


In [8]:
# number of total job 
# there are 20 job hrefs in each page
# so, if 19974 jobs / 20 ---> 997.35 pages
# but we could only get into page 150
def getPage104(href):
    res = r.get(href)
    totalPages = int(BeautifulSoup(res.text, 'lxml').select('form#jobform')[0].select('ul')[0].select('li')[0].text.split('筆')[0][1:].strip())
    res.close()
    totalPages = 150 if math.ceil(totalPages / 20)  > 150 else math.ceil(totalPages / 20) 
    return totalPages

In [9]:
# sqlite insert&update -> job104
def insert_href(title, href):
    try:
        with sqlite3.connect('job.sqlite') as conn:

            # Query histry, whether we have insert it before...
            c = conn.cursor()
            qryString = "SELECT href FROM job104 where href=:href"
            c.execute(qryString, {'href' : href})
            # if there are nothing like href
            if len(c.fetchall()) == 0:
                # insert new one
                insert_string = "INSERT INTO job104 (title, href, times) VALUES (?, ?, 1)"
                c.execute(insert_string, (title, href))
            else:
                update_string = "UPDATE job104 SET times = times + 1 WHERE href = ?"
                c.execute(update_string, (href,))

    except ConnectionError as e:
        print(e)
        print(href)

In [20]:
getPage104('https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001010%2C2007001012&order=3&asc=0&page=1')

92

In [22]:
# main program
totalPages = getPage104('https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001006&order=3&asc=0&page=1')
print('totalPages: ' + str(totalPages))
for page in range(1, totalPages + 1):
# for page in range(1, 86):
# all 
#     href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007000000&order=3&asc=0&page={}".format(page)
# 軟體專案主管
#     href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001001&order=3&asc=0&page={}".format(page)
# 通訊軟體工程師
#     href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001003&order=3&asc=0&page={}".format(page)
# 韌體設計工程師
#    href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001005&order=3&asc=0&page={}".format(page)
# 電腦系統分析師
#    href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001007&order=3&asc=0&page={}".format(page)
# 其他資訊專業人員、BIOS工程師、電子商務技術主管    
#     href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001009%2C2007001011%2C2007001002&order=3&asc=0&page={}".format(page)
# 資訊助理人員、演算法開發工程師
#    href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001010%2C2007001012&order=3&asc=0&page={}".format(page)
# Internet程式設計師
    href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001006&order=3&asc=0&page={}".format(page)
# 
#     href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=0&jobcat=2007001008&order=3&asc=0&page={}".format(page)
    
    soup = BeautifulSoup(r.get(href).text, 'lxml')
    jidSoup = soup.select('div.job_name')
    totalJid = len(jidSoup)
    print(str(page) + ' / ' + str(totalPages))
    for jid in range(0, totalJid):
        title = soup.select('div.job_name')[jid].text.strip()
        href = "https://www.104.com.tw" + jidSoup[jid].select('a')[0]['href']
        insert_href(title, href)
    
    time.sleep(1)
    
print('done!!')

totalPages: 150
1 / 150
2 / 150
3 / 150
4 / 150
5 / 150
6 / 150
7 / 150
8 / 150
9 / 150
10 / 150
11 / 150
12 / 150
13 / 150
14 / 150
15 / 150
16 / 150
17 / 150
18 / 150
19 / 150
20 / 150
21 / 150
22 / 150
23 / 150
24 / 150
25 / 150
26 / 150
27 / 150
28 / 150
29 / 150
30 / 150
31 / 150
32 / 150
33 / 150
34 / 150
35 / 150
36 / 150
37 / 150
38 / 150
39 / 150
40 / 150
41 / 150
42 / 150
43 / 150
44 / 150
45 / 150
46 / 150
47 / 150
48 / 150
49 / 150
50 / 150
51 / 150
52 / 150
53 / 150
54 / 150
55 / 150
56 / 150
57 / 150
58 / 150
59 / 150
60 / 150
61 / 150
62 / 150
63 / 150
64 / 150
65 / 150
66 / 150
67 / 150
68 / 150
69 / 150
70 / 150
71 / 150
72 / 150
73 / 150
74 / 150
75 / 150
76 / 150
77 / 150
78 / 150
79 / 150
80 / 150
81 / 150
82 / 150
83 / 150
84 / 150
85 / 150
86 / 150
87 / 150
88 / 150
89 / 150
90 / 150
91 / 150
92 / 150
93 / 150
94 / 150
95 / 150
96 / 150
97 / 150
98 / 150
99 / 150
100 / 150
101 / 150
102 / 150
103 / 150
104 / 150
105 / 150
106 / 150
107 / 150
108 / 150
109 / 150
11

In [42]:
# Query result program
# 2017/05/08 : 2960 web(exclude repeat)
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
    pprint.pprint(list(c.execute('select * from job104')))

[(1,
  '運維主管',
  'https://www.104.com.tw/job/?jobno=57o0l&jobsource=104_hotorder',
  1,
  None),
 (2,
  '系統分析師 (月薪80000元以上)',
  'https://www.104.com.tw/job/?jobno=5gm45&jobsource=104_hotorder',
  1,
  None),
 (3,
  'DevOps',
  'https://www.104.com.tw/job/?jobno=4qh2a&jobsource=104_hotorder',
  1,
  None),
 (4,
  '系統分析師 Solutions Delivery Analyst',
  'https://www.104.com.tw/job/?jobno=4cwz0&jobsource=104_hotorder',
  1,
  None),
 (5,
  '約聘系統分析師 Solutions Delivery Analyst (Contractor)',
  'https://www.104.com.tw/job/?jobno=5g0bc&jobsource=104_hotorder',
  1,
  None),
 (6,
  '資深後端工程師',
  'https://www.104.com.tw/job/?jobno=5edsi&jobsource=104_hotorder',
  2,
  None),
 (7,
  'PHP資深後端工程師 (上班地點台中)',
  'https://www.104.com.tw/job/?jobno=5f51j&jobsource=104_hotorder',
  2,
  None),
 (8,
  'Sr. 前端工程師【人才經營部】',
  'https://www.104.com.tw/job/?jobno=53nwi&jobsource=104_hotorder',
  1,
  None),
 (9,
  'Software (SW) System Design Director (DTV / STB)',
  'https://www.104.com.tw/job/?jobno=5g6yc&jobso

  None),
 (290,
  '【D720-05042】Smartphone/Tablet軟體主任工程師',
  'https://www.104.com.tw/job/?jobno=5grv9&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (291,
  '銀行(保險)業程式設計師(5/9日下班前截止收件)',
  'https://www.104.com.tw/job/?jobno=5gs1c&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (292,
  '資深軟體工程師',
  'https://www.104.com.tw/job/?jobno=5g3wt&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (293,
  'Systems Engineer',
  'https://www.104.com.tw/job/?jobno=5ey8k&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (294,
  'Network Engineer',
  'https://www.104.com.tw/job/?jobno=5eya9&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (295,
  '【DB50-4104】軟體主任工程師',
  'https://www.104.com.tw/job/?jobno=5f6gj&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (296,
  '資訊安全主管(K0600)',
  'https://www.104.com.tw/job/?jobno=5f79z&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (297,
  'Firmware Engineer',
  'https://www.104.com.tw/job/?jobno=5cx50&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (298,
  '架

  '韌體副理',
  'https://www.104.com.tw/job/?jobno=5csko&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (728,
  '(LTEG) Service QA',
  'https://www.104.com.tw/job/?jobno=5cbhc&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (729,
  '資訊部儲備幹部',
  'https://www.104.com.tw/job/?jobno=5eljk&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (730,
  '【跨國線上遊戲公司】SEO經理',
  'https://www.104.com.tw/job/?jobno=5embr&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (731,
  'C3811 Java開發 技術副理_電子數位金融平台',
  'https://www.104.com.tw/job/?jobno=5emjr&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (732,
  'Firewall Engineer - Check Point',
  'https://www.104.com.tw/job/?jobno=5emz9&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (733,
  '資訊部經理',
  'https://www.104.com.tw/job/?jobno=5ep5q&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (734,
  'Java架構師',
  'https://www.104.com.tw/job/?jobno=5epx5&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (735,
  'SAP BASIS 資深顧問',
  'https://www.104.com.tw/job/?jobno

  None),
 (1170,
  'SAP工程師(資管處)',
  'https://www.104.com.tw/job/?jobno=52zex&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1171,
  '數位EC主管/大數據主管 ( TW08849-008 ) (995)',
  'https://www.104.com.tw/job/?jobno=531m4&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1172,
  'Senior System Engineer',
  'https://www.104.com.tw/job/?jobno=53238&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1173,
  'Customer Project Manager - NRO/NDO',
  'https://www.104.com.tw/job/?jobno=535l4&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1174,
  '.NET 系統分析師/系統設計師/程式設計師 (高雄)',
  'https://www.104.com.twhttp://hunter.104.com.tw/104hunter.jsp?trc=104p&jobsource=n104bank1&urlpage=job_description&jobno=8546836',
  1,
  None),
 (1175,
  'Sr. Firmware Engineer',
  'https://www.104.com.tw/job/?jobno=539am&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1176,
  '資深軟體架構師',
  'https://www.104.com.tw/job/?jobno=53a8e&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1177,
  'Java 主任工程師 (台北辦事處)',
  'https://w

  '專案經理 \u200b\u200b\u200b',
  'https://www.104.com.tw/job/?jobno=2uc0y&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1668,
  '大陸廠資訊主管(中山)',
  'https://www.104.com.tw/job/?jobno=2x7cz&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1669,
  'Sr. Java Programmer / Manager',
  'https://www.104.com.tw/job/?jobno=30q35&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1670,
  '元大證券-資訊部-資訊工程師',
  'https://www.104.com.tw/job/?jobno=31kwc&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1671,
  'BIOS 資深工程師',
  'https://www.104.com.tw/job/?jobno=31vca&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1672,
  'Server UEFI/BIOS leader(桃園)',
  'https://www.104.com.tw/job/?jobno=32a8z&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1673,
  'SAP 資深顧問',
  'https://www.104.com.tw/job/?jobno=32voh&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1674,
  'SAP 專案顧問經理/主管(PP, CRM)',
  'https://www.104.com.tw/job/?jobno=33439&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (1675,
  '高級軟體設計工程師 (Li

  'https://www.104.com.tw/job/?jobno=5fxxt&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2153,
  'MIS主管',
  'https://www.104.com.tw/job/?jobno=5fy0b&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2154,
  '大數據分析師',
  'https://www.104.com.tw/job/?jobno=5fy5e&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2155,
  '數位行銷經理',
  'https://www.104.com.tw/job/?jobno=5fyah&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2156,
  '(台北)系統分析師．資訊研發服務部雲端服務處(外勤筋斗雲系統)',
  'https://www.104.com.tw/job/?jobno=5fyk7&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2157,
  '(台中)系統分析師．資訊研發服務部雲端服務處(外勤筋斗雲系統)',
  'https://www.104.com.tw/job/?jobno=5fyo2&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2158,
  '【Operation】System Analyst(美國)',
  'https://www.104.com.tw/job/?jobno=5fyrt&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2159,
  '【資訊】整合行銷處大數據工程師',
  'https://www.104.com.tw/job/?jobno=5fz20&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2160,
  '【家教】作業系統-社會人士',
  'https://www.104.com.t

  'https://www.104.com.tw/job/?jobno=59g1u&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2622,
  '車用軟體工程師5-UI engineer',
  'https://www.104.com.tw/job/?jobno=59g2b&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2623,
  'UI Designer(Web)(高雄分公司)',
  'https://www.104.com.tw/job/?jobno=59g8c&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2624,
  '資深網頁前端工程師(台北總公司)',
  'https://www.104.com.tw/job/?jobno=59g8l&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2625,
  '資訊處副理',
  'https://www.104.com.tw/job/?jobno=59gi7&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2626,
  'A8 NOC 資深網管工程師',
  'https://www.104.com.tw/job/?jobno=59h0x&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2627,
  'VE3 規設網路系統工程師',
  'https://www.104.com.tw/job/?jobno=59h1g&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2628,
  '網路工程師(駐點人員)',
  'https://www.104.com.tw/job/?jobno=59hdk&jobsource=n104bank1&hotjob_chr=',
  1,
  None),
 (2629,
  '軟體測試工程師',
  'https://www.104.com.tw/job/?jobno=59htb&jobsourc

In [48]:
# Query maximun times
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
#     pprint.pprint(list(c.execute('select * from job104 where times > 1')))
    pprint.pprint(list(c.execute('select count(times), sum(times) from job104 where times > 1')))

[(19, 69)]


In [ ]:
# ranking function
# ranking = sorted(dict.items(),key=lambda d:d[1],reverse = True) #用出現次數排順序